In [2]:
import os
import sys

# Adiciona a pasta 'src' ao sys.path
sys.path.append(os.path.abspath(os.path.join("../..", "src")))

In [3]:
from datetime import datetime

import duckdb
import polars as pl
from loguru import logger

In [6]:
from modulos.contrato_de_dados import contrato_entrada, contrato_saida
from modulos.uteis import (
    carregar_env,
    configuracao_loguru,
    configuracao_polars,
)

In [7]:
configuracao_loguru.configuracao_loguru()
configuracao_polars.configuracao_polars("../../config/polars.json")

In [8]:
df = pl.read_json("../../dados/nao_processados/mercado_livre.json")

df

marca,nome,preco_velho_reais,preco_velho_centavos,preco_novo_reais,preco_novo_centavos,nota_avaliacao,num_avaliacoes
str,str,str,str,str,str,str,str
"""ADIDAS""","""Tênis Masculino Duramo Sl adid…","""499""","""99""","""356""","""35""",null,null
"""RAYCONN""","""Tenis 4 Molas Masculino Corrid…","""186""","""95""","""164""","""51""","""4.3""","""(74)"""
"""VILI""","""Tênis Masculino Academia Camin…","""108""","""39""","""10""","""54""","""4.5""","""(2488)"""
"""FILA""","""Tênis Fila Improve Masculino P…","""399""","""90""","""39""","""99""",null,null
"""FILA""","""Tênis Fila Venture Tracer Masc…","""439""","""98""","""42""","""66""",null,null
…,…,…,…,…,…,…,…
null,"""Tênis Okley Na Promoção Bravo …","""109""","""90""","""36""","""63""","""4.5""","""(95)"""
"""OLYMPIKUS""","""Olympikus Cyber 3 Masculino Ad…","""197""","""99""","""19""","""24""","""4.7""","""(43)"""
"""OLYMPIKUS""","""Olympikus CORRIDA Swift 4 Femi…","""279""","""99""","""31""","""11""","""4.8""","""(1115)"""


In [5]:
# Adicionando novas colunas
# - Preco_Velho
# Caso preco_velho_real e preco_velho_centavos nao sejam null -> "preco_velho_real" + "." + "preco_velho_centavos"
# Caso preco_velho_real seja null -> null
# Caso preco_velho_centavo seja null -> "preco_velho_real"
# - Preco_Novo
# Mesma coisa do Preco_Velho

In [6]:
df.select(
    pl.col("preco_velho_reais"),
    pl.col("preco_velho_centavos"),
    pl.when(
        (pl.col("preco_velho_reais").is_not_null())
        & (pl.col("preco_velho_centavos").is_not_null())
    )
    .then(
        pl.col("preco_velho_reais").str.replace(".", "", literal=True)
        + "."
        + pl.col("preco_velho_centavos")
    )
    .otherwise(
        pl.when(pl.col("preco_velho_centavos").is_null())
        .then(pl.col("preco_velho_reais").str.replace(".", "", literal=True))
        .otherwise(None)
    )
    .alias("preco_velho")
    .cast(pl.Float32),
)

preco_velho_reais,preco_velho_centavos,preco_velho
str,str,f32
"""499""","""99""","499,99"
"""186""","""95""","186,95"
"""108""","""39""","108,39"
"""399""","""90""","399,90"
"""439""","""98""","439,98"
…,…,…
"""109""","""90""","109,90"
"""197""","""99""","197,99"
"""279""","""99""","279,99"


In [7]:
df.select(
    pl.col("preco_novo_reais"),
    pl.col("preco_novo_centavos"),
    pl.when(
        (pl.col("preco_novo_reais").is_not_null())
        & (pl.col("preco_novo_centavos").is_not_null())
    )
    .then(
        pl.col("preco_novo_reais").str.replace(".", "", literal=True)
        + "."
        + pl.col("preco_novo_centavos")
    )
    .otherwise(
        pl.when(pl.col("preco_novo_centavos").is_null())
        .then(pl.col("preco_novo_reais").str.replace(".", "", literal=True))
        .otherwise(None)
    )
    .alias("preco_novo")
    .cast(pl.Float32),
)

preco_novo_reais,preco_novo_centavos,preco_novo
str,str,f32
"""356""","""35""","356,35"
"""164""","""51""","164,51"
"""10""","""54""","10,54"
"""39""","""99""","39,99"
"""42""","""66""","42,66"
…,…,…
"""36""","""63""","36,63"
"""19""","""24""","19,24"
"""31""","""11""","31,11"


In [26]:
data = datetime.now()
ano = data.year
mes = data.month
dia = data.day
hora = data.hour
minuto = data.minute
segundo = data.second

In [27]:
df.select(pl.datetime(ano, mes, dia, hora, minuto, segundo))

datetime
datetime[μs]
2024-08-07 12:22:39


In [16]:
df.select(
    pl.lit("https://lista.mercadolivre.com.br/tenis-corrida-masculino").alias(
        "_fonte"
    )
)

_fonte
str
"""https://lista.mercadolivre.com…"


In [28]:
df.with_columns(
    pl.lit("https://lista.mercadolivre.com.br/tenis-corrida-masculino").alias(
        "_fonte"
    ),
    pl.datetime(ano, mes, dia, hora, minuto, segundo).alias("_data_coleta"),
)

marca,nome,preco_velho_reais,preco_velho_centavos,preco_novo_reais,preco_novo_centavos,nota_avaliacao,num_avaliacoes,_fonte,datetime
str,str,str,str,str,str,str,str,str,datetime[μs]
"""ADIDAS""","""Tênis Masculino Duramo Sl adid…","""499""","""99""","""356""","""35""",null,null,"""https://lista.mercadolivre.com…",2024-08-07 12:22:39
"""RAYCONN""","""Tenis 4 Molas Masculino Corrid…","""186""","""95""","""164""","""51""","""4.3""","""(74)""","""https://lista.mercadolivre.com…",2024-08-07 12:22:39
"""VILI""","""Tênis Masculino Academia Camin…","""108""","""39""","""10""","""54""","""4.5""","""(2488)""","""https://lista.mercadolivre.com…",2024-08-07 12:22:39
"""FILA""","""Tênis Fila Improve Masculino P…","""399""","""90""","""39""","""99""",null,null,"""https://lista.mercadolivre.com…",2024-08-07 12:22:39
"""FILA""","""Tênis Fila Venture Tracer Masc…","""439""","""98""","""42""","""66""",null,null,"""https://lista.mercadolivre.com…",2024-08-07 12:22:39
…,…,…,…,…,…,…,…,…,…
null,"""Tênis Okley Na Promoção Bravo …","""109""","""90""","""36""","""63""","""4.5""","""(95)""","""https://lista.mercadolivre.com…",2024-08-07 12:22:39
"""OLYMPIKUS""","""Olympikus Cyber 3 Masculino Ad…","""197""","""99""","""19""","""24""","""4.7""","""(43)""","""https://lista.mercadolivre.com…",2024-08-07 12:22:39
"""OLYMPIKUS""","""Olympikus CORRIDA Swift 4 Femi…","""279""","""99""","""31""","""11""","""4.8""","""(1115)""","""https://lista.mercadolivre.com…",2024-08-07 12:22:39
